# prepare_training_data
**Author:** Khoi Nguyen

**Date created:** 03/06/2023

**Last modified:** 04/15/2023

**Description:** This notebook is used to create the response-completion pairs for training the `Ada` models. This task is accomplished by using the `test` sets w/ the "Improve this sentence:" prompt and submitting that to `Curie` to create synthetic datasets.

**WARNING:** This notebook requires API calls and will cost money. Please be careful when running this step.

In [1]:
import json
import openai
import os
import tqdm

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
instruction = "Elaborate on the following sentence: "

### 1k_ada
Synthesizing `1k_ada` training data.

In [ ]:
confirmation = input('Are you sure you want to run 1k_ada synthethic training creation? Type YES or NO: ')

if confirmation == 'YES':
    with open('data/1k_ada/train.json') as f:
        train_data = json.load(f)

    with open('data/1k_ada/openai_training.jsonl', 'w') as outfile:
        for sentence in tqdm.tqdm(train_data):
            prompt = "{instruction}\"{sentence}\"\n\n".format(sentence=sentence["sentence"], instruction=instruction)
            response = openai.Completion.create(
                model="text-curie-001",
                prompt=prompt,
                temperature=0.7,
                max_tokens=256,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
            )
            outfile.write(json.dumps({
                'prompt': prompt,
                'completion': response.choices[0].text
            }) + '\n')
            outfile.flush()
else:
    print('Synthethic training creation aborted.')

In [ ]:
# Load in openai_training.jsonl and print the first 10 lines
with open('data/1k_ada/openai_training.jsonl') as f:
    for i in range(10):
        print(f.readline())

In [ ]:
# Each prompt should end with a fixed separator to inform the model when the prompt ends and the completion begins. A simple separator which generally works well is \n\n###\n\n. The separator should not appear elsewhere in any prompt.
# Each completion should start with a whitespace due to our tokenization, which tokenizes most words with a preceding whitespace.
# Each completion should end with a fixed stop sequence to inform the model when the completion ends. A stop sequence could be \n, ###, or any other token that does not appear in any completion.
# For inference, you should format your prompts in the same way as you did when creating the training dataset, including the same separator. Also specify the same stop sequence to properly truncate the completion.

with open('data/1k_ada/openai_training.jsonl', 'r') as f:
    with open('data/1k_ada/openai_cleaned.jsonl', 'w') as out:
        for line in f:
            data = json.loads(line)
            prompt = data['prompt'].strip() + '\n\n##\n\n'
            completion = ' ' + data['completion'].strip() + '\n'
            out.write(json.dumps({'prompt': prompt, 'completion': completion}))
            out.write('\n')
            

In [ ]:
# Load in openai_cleaned.jsonl and print the first 5 lines with prettier formatting
with open('data/1k_ada/openai_cleaned.jsonl') as f:
    for i in range(5):
        print(json.dumps(json.loads(f.readline()), indent=4))

In [4]:
# Run the following command in the terminal to use the OpenAI CLI Data Preparation Tool to prepare the data for training.
# In addition to the cleaning processes above, the tool will remove the prompt instruction
# ! openai tools fine_tunes.prepare_data -f data/1k_ada/openai_cleaned.jsonl

### 10k_ada
Synthesizing `10k_ada` training data.

In [2]:
confirmation = input('Are you sure you want to run 10k_ada synthethic training creation? Type YES or NO: ')

if confirmation == 'YES':
    with open('data/10k_ada/train.json') as f:
        train_data = json.load(f)

    with open('data/10k_ada/openai_training.jsonl', 'w') as outfile:
        for sentence in tqdm.tqdm(train_data):
            prompt = "{instruction}\"{sentence}\"\n\n".format(sentence=sentence["sentence"], instruction=instruction)
            response = openai.Completion.create(
                model="text-curie-001",
                prompt=prompt,
                temperature=0.7,
                max_tokens=256,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
            )
            outfile.write(json.dumps({
                'prompt': prompt,
                'completion': response.choices[0].text
            }) + '\n')
            outfile.flush()
else:
    print('Synthethic training creation aborted.')

100%|██████████| 10000/10000 [1:41:43<00:00,  1.64it/s]


In [3]:
# Load in openai_training.jsonl and print the first 10 lines
with open('data/10k_ada/openai_training.jsonl') as f:
    for i in range(10):
        print(f.readline())

{"prompt": "Elaborate on the following sentence: \"All jet engines have a combustor or burner in which the air and fuel are mixed and burned.\"\n\n", "completion": "\n\nJet engines typically have a combustor or burner in which the air and fuel are mixed and burned. This mixture is then forced into the engine's exhaust pipe where it is burned to create thrust."}

{"prompt": "Elaborate on the following sentence: \"Melanin is what darkens the skin, producing a suntan.\"\n\n", "completion": "\n\nMelanin is a pigment found in the skin that produces a tan."}

{"prompt": "Elaborate on the following sentence: \"Nutritious snacks are smart ways to get the nutrients and energy needed to work, play and grow.\"\n\n", "completion": "\n\nA nutritious snack can be anything that provides nutrients and energy, such as a piece of fruit, a yogurt, a granola bar, or a piece of dark chocolate. Snacks can be a great way to get the nutrients and energy you need to function optimally. They can also help you t

In [4]:
# Each prompt should end with a fixed separator to inform the model when the prompt ends and the completion begins. A simple separator which generally works well is \n\n###\n\n. The separator should not appear elsewhere in any prompt.
# Each completion should start with a whitespace due to our tokenization, which tokenizes most words with a preceding whitespace.
# Each completion should end with a fixed stop sequence to inform the model when the completion ends. A stop sequence could be \n, ###, or any other token that does not appear in any completion.
# For inference, you should format your prompts in the same way as you did when creating the training dataset, including the same separator. Also specify the same stop sequence to properly truncate the completion.

with open('data/10k_ada/openai_training.jsonl', 'r') as f:
    with open('data/10k_ada/openai_cleaned.jsonl', 'w') as out:
        for line in f:
            data = json.loads(line)
            prompt = data['prompt'].strip() + '\n\n##\n\n'
            completion = ' ' + data['completion'].strip() + '\n'
            out.write(json.dumps({'prompt': prompt, 'completion': completion}))
            out.write('\n')
            

In [ ]:
# Run the following command in the terminal to use the OpenAI CLI Data Preparation Tool to prepare the data for training.
# In addition to the cleaning processes above, the tool will remove the prompt instruction
# ! openai tools fine_tunes.prepare_data -f data/10k_ada/openai_cleaned.jsonl

### 100k_ada
Synthesizing `100k_ada` training data.

In [ ]:
confirmation = input('Are you sure you want to run 100k_ada synthethic training creation? Type YES or NO: ')

if confirmation == 'YES':
    with open('data/100k_ada/train.json') as f:
        train_data = json.load(f)

    with open('data/100k_ada/openai_training.jsonl', 'w') as outfile:
        for sentence in tqdm.tqdm(train_data):
            prompt = "{instruction}\"{sentence}\"\n\n".format(sentence=sentence["sentence"], instruction=instruction)
            response = openai.Completion.create(
                model="text-curie-001",
                prompt=prompt,
                temperature=0.7,
                max_tokens=256,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
            )
            outfile.write(json.dumps({
                'prompt': prompt,
                'completion': response.choices[0].text
            }) + '\n')
            outfile.flush()
else:
    print('Synthethic training creation aborted.')

In [ ]:
# Load in openai_training.jsonl and print the first 10 lines
with open('data/100k_ada/openai_training.jsonl') as f:
    for i in range(10):
        print(f.readline())

In [ ]:
# Each prompt should end with a fixed separator to inform the model when the prompt ends and the completion begins. A simple separator which generally works well is \n\n###\n\n. The separator should not appear elsewhere in any prompt.
# Each completion should start with a whitespace due to our tokenization, which tokenizes most words with a preceding whitespace.
# Each completion should end with a fixed stop sequence to inform the model when the completion ends. A stop sequence could be \n, ###, or any other token that does not appear in any completion.
# For inference, you should format your prompts in the same way as you did when creating the training dataset, including the same separator. Also specify the same stop sequence to properly truncate the completion.

with open('data/100k_ada/openai_training.jsonl', 'r') as f:
    with open('data/100k_ada/openai_cleaned.jsonl', 'w') as out:
        for line in f:
            data = json.loads(line)
            prompt = data['prompt'].strip() + '\n\n##\n\n'
            completion = ' ' + data['completion'].strip() + '\n'
            out.write(json.dumps({'prompt': prompt, 'completion': completion}))
            out.write('\n')
            

In [ ]:
# Run the following command in the terminal to use the OpenAI CLI Data Preparation Tool to prepare the data for training.
# In addition to the cleaning processes above, the tool will remove the prompt instruction
# ! openai tools fine_tunes.prepare_data -f data/100k_ada/openai_cleaned.jsonl